In [1]:
import numpy as np
import pandas as pd
from numpy import array
from math import sqrt
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
import matplotlib.pyplot as plt

from pyspark.sql.functions import col


ModuleNotFoundError: No module named 'pyspark'

SyntaxError: invalid syntax (<ipython-input-20-26117a1dc40e>, line 1)

In [ ]:
spark.version#checking sc version
sqlContext = SQLContext(sc) # taking sqlcontext for reading data
#importing statistics of batsman and bowlers
batting=sqlContext.read.csv("./../Data/stats_batting_2.csv", header=True, mode="DROPMALFORMED")#reading csv file
bowling=sqlContext.read.csv("./../Data/stats_bowling_1 .csv", header=True, mode="DROPMALFORMED")
bowling.show()#displaying bowling rdd
batting.show()#displaying batting rdd

In [ ]:
#preprocess the data -- cleaning the data -- removing null values
batting.na.drop()
bowling=bowling.na.drop()


#selecting feature columns in batting table and bowling table 
#these features are used for clustering
#we have used all columns for clustering.
FEATURES_COL=["runs_scored","balls_faced","times_out","batting_average","strike_rate"]
FEATURES_COL_BOWL=["runs_conceded","wickets_taken","overs_bowled","bowling_average","economy_rate","bowling_strike_rate"]


In [ ]:
#converting the columns valus to  float 
for col in batting.columns:
    if col in FEATURES_COL:
        batting = batting.withColumn(col,batting[col].cast('float'))
batting.show()

for col in bowling.columns:
    if col in FEATURES_COL_BOWL:
        bowling = bowling.withColumn(col,bowling[col].cast('float'))
bowling.show()

In [2]:
#transforming the table into two column dataframe 
# creating a list of features for passing it to kmeans function
vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
df_kmeans = vecAssembler.transform(batting).select('player_name', 'features')
df_kmeans.show()

vecAssembler_bowl = VectorAssembler(inputCols=FEATURES_COL_BOWL, outputCol="features_bowl")
df_kmeans_bowl = vecAssembler_bowl.transform(bowling).select('player_name', 'features_bowl')
df_kmeans_bowl.show()


NameError: name 'VectorAssembler' is not defined

In [3]:
# for finding the number of clusters using wcss or cost method in ,llib
# for batsman
cost = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df_kmeans.sample(False,0.1, seed=42))
    cost[k] = model.computeCost(df_kmeans) # requires Spark 2.0 or later

#     for bowler
cost_bowl = np.zeros(20)
for k in range(2,20):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features_bowl")
    model = kmeans.fit(df_kmeans_bowl.sample(False,0.1, seed=42))
    cost_bowl[k] = model.computeCost(df_kmeans_bowl) # requires Spark 2.0 or later

NameError: name 'KMeans' is not defined

In [4]:
# plotting the graph of cost method for both batsman and bowler
fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,20),cost[2:20])
ax.set_xlabel('k')
ax.set_ylabel('cost')
# we selected 10 cluster as there is sudden change in graph at 10

fig, ax = plt.subplots(1,1, figsize =(8,6))
ax.plot(range(2,20),cost_bowl[2:20])
ax.set_xlabel('k')
ax.set_ylabel('cost_bowl')
# we selected 10 cluster as there is sudden change in graph at 7

NameError: name 'plt' is not defined

In [5]:
k = 10 #cluster number for batsman
kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features") # trainfing model 
model = kmeans.fit(df_kmeans) #fitting the model for df_kmeans table which is batsmans
centers = model.clusterCenters()

print("Cluster Centers: ")
for center in centers:
    print(center)#printing centres for batsman clusters

k = 7 #cluster number for bowlwer
kmeans_bowl = KMeans().setK(k).setSeed(1).setFeaturesCol("features_bowl")#training model
model_bowl = kmeans_bowl.fit(df_kmeans_bowl)#fitting the model for df_kmeans table which is batsmans
centers_bowl = model_bowl.clusterCenters()

print("Cluster Centers: ")
for center in centers_bowl:
    print(center)#printing centres for batsman clusters
    

NameError: name 'KMeans' is not defined

In [6]:
#slecting the columns using transform operation
transformed = model.transform(df_kmeans).select('player_name', 'prediction')
#collecting data from all the nodes
rows = transformed.collect()
#printing first 3 rows
print(rows[:3])

#slecting the columns using transform operation
transformed = model_bowl.transform(df_kmeans_bowl).select('player_name', 'prediction')
rows_bowl = transformed.collect()
print(rows_bowl[:3])

NameError: name 'model' is not defined

In [ ]:
#creating sqldataframe to work on that
df_pred = sqlContext.createDataFrame(rows)
df_pred.show()

df_pred_bowl = sqlContext.createDataFrame(rows_bowl)
df_pred_bowl.show()

In [7]:
# converting swl dataframe to pandas dataframe
bat_cluster=df_pred.toPandas()
bat_cluster.to_csv("./../Data/batting_cluster.csv")

bowl_cluster=df_pred_bowl.toPandas()
bowl_cluster.to_csv("./../Data/bowling_cluster.csv")


NameError: name 'df_pred' is not defined

In [8]:
# df_pred = df_pred.join(batting, 'player_name')
# df_pred_bowl = df_pred_bowl.join(bowling, 'player_name')
df_pred.show()
df_pred_bowl.show()


NameError: name 'df_pred' is not defined

In [9]:
# reading runs probability and wicket probability
batting_prob=sqlContext.read.csv("./../Data/r_prob.csv", header=True, mode="DROPMALFORMED")
bowling_prob=sqlContext.read.csv("./../Data/wicket.csv",header=True,mode="DROPMALFORMED")
# batting_prob.show()
# bowling_prob.show()


NameError: name 'sqlContext' is not defined

In [10]:
# this code is for merging tables in sql context

batting_prob_clus=batting_prob.join(df_pred_bowl,df_pred_bowl.player_name==batting_prob.bowler,'leftouter').select(batting_prob["*"],df_pred_bowl["prediction"])

batting_prob_clus=batting_prob_clus.selectExpr("_c0 as index","batsman as batsman","bowler as bowler","P0 as P0","P1 as P1","P2 as P2","P3 as P3","P4 as P4","P6 as P6","prediction as bowler_cluster")
batting_prob_clus=batting_prob_clus.join(df_pred,df_pred.player_name==batting_prob_clus.batsman,'leftouter').select(batting_prob_clus["*"],df_pred["prediction"])
batting_prob_clus=batting_prob_clus.selectExpr("index as index","batsman as batsman","bowler as bowler","P0 as P0","P1 as P1","P2 as P2","P3 as P3","P4 as P4","P6 as P6","bowler_cluster as bowler_cluster","prediction as batsman_cluster")


bowling_prob_clus=bowling_prob.join(df_pred_bowl,df_pred_bowl.player_name==bowling_prob.bowler,'leftouter').select(bowling_prob["*"],df_pred_bowl["prediction"])
bowling_prob_clus=bowling_prob_clus.selectExpr("_c0 as index","batsman as batsman","bowler as bowler","no_of_times as no_of_times","no_of_balls as no_of_balls","p_out as p_out","p_not_out as p_not_out","prediction as bowler_cluster")
bowling_prob_clus=bowling_prob_clus.join(df_pred,df_pred.player_name==bowling_prob_clus.batsman,'leftouter').select(bowling_prob_clus["*"],df_pred["prediction"])
bowling_prob_clus=bowling_prob_clus.selectExpr("index as index","batsman as batsman","bowler as bowler","no_of_times as no_of_times","no_of_balls as no_of_balls","p_out as p_out","p_not_out as p_not_out","bowler_cluster as bowler_cluster","prediction as batsman_cluster")
batting_prob_clus.show()


NameError: name 'batting_prob' is not defined

In [11]:
# transforming them to pandas
pd_bowling_prob=bowling_prob_clus.toPandas()
pd_batting_prob=batting_prob_clus.toPandas()

NameError: name 'bowling_prob_clus' is not defined

In [12]:
# storing into csv file
pd_batting_prob.to_csv("./../Data/pd_batting_prob.csv")
pd_bowling_prob.to_csv("./../Data/pd_bowling_prob.csv")


NameError: name 'pd_batting_prob' is not defined

In [13]:
# reading from csv file
df=pd.read_csv('./../Data/pd_batting_prob.csv')
df2=pd.read_csv('./../Data/pd_bowling_prob.csv')


In [14]:
# for getting cluster_prob.csv
# grouping the data of pp prob and selecting bowler and batsman cluster column and their run prob and grouping by averaging their prob
res = df.groupby(["bowler_cluster","batsman_cluster"])['P0','P1','P2','P3','P4','P6'].mean().round(2).reset_index()
# grouping the data of pp prob and selecting bowler and batsman cluster column and their not_out prob and grouping by averaging their prob
res2 = df2.groupby(["bowler_cluster","batsman_cluster"])['p_not_out'].mean().round(2).reset_index()

In [15]:
# for getting pp_prob.csv
# grouping the data of pp prob and selecting bowler and batsman column and their run prob and grouping by averaging their prob
ref = df.groupby(["batsman","bowler"])['P0','P1','P2','P3','P4','P6'].mean().round(2).reset_index()
ref2 = df2.groupby(["batsman","bowler"])['p_not_out'].mean().round(2).reset_index()

In [16]:
# merging two tables
final_ref=ref
final_ref=final_ref.merge(ref2,how='outer',left_on=["batsman","bowler"],right_on=["batsman","bowler"])

In [17]:
# merging two tables
final=res
final['p_not_out']=res2['p_not_out']

In [18]:
final_ref.to_csv('./../Data/pp_prob.csv',index=None)
final.to_csv('./../Data/clusters_prob.csv',index=None)

# found pp_prob and cluster prob

In [19]:
#--------------------------